In [1]:
import numpy as np
import pandas as pd

In [5]:
# dataset coronavirus WHO
pd.set_option('max_colwidth', 200)  # Increase column width
data = pd.read_excel("BASE_QA.xlsx")
data.head()

,Context,Answer
0,What are the Sustainable Development Goals?,"The 17 SDGs, also known as the Global Goals, are part of a comprehensive agenda framing global sustainable development up to 2030. The SDGs build upon and extend the Millennium Development Goals (..."
1,What is special about the 2030 Agenda?,"The 2030 Agenda is a bold call for action, setting out a vision and principles for sustainable development in the coming decade. The 2030 Agenda was adopted in 2015 at the United Nations summit in..."
2,How is health included in the 2030 Agenda?,"The 2030 Agenda provides for strong political commitment to public health. It states that “To promote physical and mental health and well-being, and to extend life expectancy for all, we must achi..."
3,Is there a specific SDG regarding health?,Yes. Goal 3.
4,How is progress towards SDGs measured?,"Commitment to the SDGs is intensifying the need for strengthened national and subnational systems for integrated monitoring, including of health programmes and performance. Maintaining momentum to..."


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import re

In [ ]:
def preprocess_sentences(input_sentences):
    return [re.sub(r'(covid-19|covid)', 'coronavirus', input_sentence, flags=re.I) 
            for input_sentence in input_sentences]
        
# Load module containing USE
module = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-qa/3')

# Create response embeddings
response_encodings = module.signatures['response_encoder'](
        input=tf.constant(preprocess_sentences(data.Answer)),
        context=tf.constant(preprocess_sentences(data.Context)))['outputs']